## Analysing Results for D3D

A workflow for reading and visualizing data.

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
from pyPoseidon.utils import data,point
from pyPoseidon.utils.vis import video
import numpy as np
import xarray as xr
import glob
import hvplot.pandas
import hvplot.xarray

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

## Analysis of output 

In [ ]:
PATH = '/Users/brey/Downloads/EUR/D3D/20180901.00/'

In [ ]:
folders = glob.glob(PATH + '*')
folders

In [ ]:
folders=[PATH]
folders

In [ ]:
otp = data(**{'solver':'schism','folders': folders, 'tag' : 'eur01'})#,'savenc':True})#,tag='eur025')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.impl.Dataset

In [ ]:
otp.impl.dem.bathymetry.hvplot(x='longitude', y='latitude', width=800 , height=600,cmap='viridis')

In [ ]:
X, Y = np.meshgrid(otp.impl.Dataset.XZ.values,otp.impl.Dataset.YZ.values)
xh = np.ma.masked_array(X, otp.impl.w) #mask land
yh = np.ma.masked_array(Y, otp.impl.w)

In [ ]:
#fig = plt.figure(figsize=(12,8))
#ax = plt.subplot(projection=ccrs.PlateCarree());

#gp = ax.scatter(otp.impl.grid.lons, otp.impl.grid.lats, s=2, color='k',  label='grid points', transform=ccrs.PlateCarree());

#pp = ax.scatter(xh, yh, s=2, color='b', label='pressure points', transform=ccrs.PlateCarree());

#ax.legend()

#ax.coastlines('50m'); ax.gridlines(draw_labels=True);

In [ ]:
grid = xr.Dataset({    'lons': (['latitude', 'longitude'], xh),   
                       'lats': (['latitude', 'longitude'], yh)},   
                          coords={'longitude': ('longitude', otp.impl.Dataset.XZ.values),   
                                  'latitude': ('latitude', otp.impl.Dataset.YZ.values),   
                                  })   

grid

In [ ]:
grid[['lons','lats']].to_dataframe().hvplot.points(x='lons',y='lats', geo=True, datashade=True, width=800, height=600, hover=False) * gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
#grid.hvplot.points(x='lons',y='lats', geo=True, datashade=True, width=800, height=600) * gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

### Visualize

In [ ]:
otp.impl.Dataset.hvplot.quadmesh(x='XZ', y='YZ', z='S1', width=800, height=600,cmap='viridis', dynamic=True, rasterize=True, clim=(-1., 1.))#, geo=True) #* gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
otp.impl.Dataset.S1.hvplot(x='XZ',y='YZ',width=800, height=600,cmap='viridis', geo=True, dynamic=True, rasterize=True, clim=(-1., 1.), projection=ccrs.PlateCarree()) * gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
otp.impl.Dataset.S1.isel(time=10).hvplot.contour(x='XZ',y='YZ',width=800, height=600,cmap='viridis', levels = 30)

In [ ]:
otp.impl.Dataset.S1.isel(time=10).hvplot.contourf(x='XZ',y='YZ',width=800, height=600,cmap='viridis', levels = 30)

In [ ]:
otp.impl.Dataset.S1.hvplot.line(width=800, height=600)

In [ ]:
v = otp.impl.frames(['S1'],title='SSH', nv =20 )#,vmin=-.5, vmax=.5)
v

In [ ]:
v.save('/Users/brey/Downloads/ssh.mp4', writer='ffmpeg')

In [ ]:
video('/Users/brey/Downloads/ssh.mp4','mp4') # optional show mp4 video

In [ ]:
w = otp.impl.frames(['U1Z','V1Z'],title='Vel')#,scale=.1)
w

In [ ]:
w.save('/Users/brey/Downloads/uv.mp4', writer='ffmpeg')

In [ ]:
video('/Users/brey/Downloads/uv.mp4','mp4') # optional show mp4 video

## Observation points / Validation

In [ ]:
otp.impl.info['tag']

#### Numerical simulation time series

In [ ]:
otp.impl.info['obs'].head()

In [ ]:
files = [folder +'/trih-'+otp.impl.info['tag']+'.nc' for folder in folders]
files

In [ ]:
op = xr.open_mfdataset(files)

In [ ]:
XS = op.XSTAT[0,:].values
YS = op.YSTAT[0,:].values

In [ ]:
op

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

otp.impl.dem.bathymetry.plot(ax=ax)

ax.scatter(XS,YS,marker='X')

ax.coastlines('50m'); ax.gridlines(draw_labels=True);

In [ ]:
op

In [ ]:
op.ZWL[:,0]

#### Tide gauge data

In [ ]:
otp.impl.obs.locations

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(0)
print (p.head())
print (p.tail())

In [ ]:
#get with Station Name  
p = otp.impl.obs.loc('Denmark - Faaborg')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
plat,plon = otp.impl.obs.locations.loc[0,['lat','lon']]
plat, plon

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

gp = ax.contourf(otp.impl.dem.longitude, otp.impl.dem.latitude, otp.impl.dem.bathymetry, transform=ccrs.PlateCarree());

pp = ax.scatter(plon, plat, marker='x', s=50, color='r', label='obs locations', transform=ccrs.PlateCarree());



In [ ]:
#ts = otp.vars.S1.sel(XZ=[lon], method='nearest', tolerance=5).sel(YZ=[lat], method='nearest', tolerance=5)
#ts = ts.squeeze().to_pandas()
#ts.head()

In [ ]:
ts = point(lon=plon,lat=plat,data=otp.impl)
ts.tseries(var='S1')

In [ ]:
ts.S1.head()

In [ ]:
ts.S1.plot()

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.S1.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['D3D'], loc='center left',bbox_to_anchor=(1.1, 0.5))

...or...

In [ ]:
## Join the graphs
p.hvplot(figsize=(14,10)) * ts.S1.hvplot(label='d3d',color='red')